<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

- git lfs install
- git clone https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis

In [13]:
!git lfs install
!git clone https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
!pip install transformers

Updated git hooks.
Git LFS initialized.
fatal: destination path 'distilroberta-finetuned-financial-news-sentiment-analysis' already exists and is not an empty directory.


In [14]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
# cur_path = "/content/drive/MyDrive/Colab Notebooks/capstone/"
# os.chdir(cur_path)
# !pwd

In [15]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from transformers import pipeline
import time
import datetime

In [16]:
def load_data(): 
    path = r'/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/'
    #path = r'/content/drive/MyDrive/Colab Notebooks/capstone/gdelt_data/'
    years = os.listdir(path)
    years.remove('.DS_Store')
    years.remove('.ipynb_checkpoints')
    yr = [x for x in years if not (x.startswith('.'))]

    year_paths= []
    months=[]
    file_paths= []

    for i in range(len(years)): # enter year folder
        year_paths.append(path + years[i]) # year paths
        months.extend(os.listdir(year_paths[i])) # months of data in each year
    months = [x for x in months if not (x.startswith('.'))]
    for j in range(len(months)): # enter months in each year folder
        file_paths.append(year_paths[i] + "/" + months[j]) # file in year path
    
    return(file_paths)

In [17]:
def filter_websites(file):
    df = pd.read_csv(file, index_col=[0]) # read in csv
    m_df = df[df.website=='marketwatch.com'] # filter for website
    p_df = df[df.website=='prnewswire.com'] # filter for website
    y_df = df[df.website=='yahoo.com'][0:10] # filter for website
    
    return(m_df, p_df, y_df)

In [18]:
roberta = pipeline(task='sentiment-analysis', 
                   model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
                   return_all_scores=True,
                   max_length=514, 
                   truncation=True)

In [19]:
def summary_stats(score):
    avg = np.mean(score)
    med = np.median(score)
    min = np.min(score)
    max = np.max(score)
    return(avg, med, min, max)

In [20]:
# yahoo web scrape NLP
def yahoo_nlp(urls, df):

    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    nf = "Content is currently unavailable"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
                continue
        try:
            try:
                master = soup.find(id="Masterwrap")
                final = master.find("div", class_="caas-body")
            except AttributeError:
                master = soup.find(id="Masterwrap2Col")
                final = master.find("div", class_="caas-body")
        except AttributeError:
            pass
        text = re.sub('<[^>]+>', '', str(final))
        sen_list = text.split(".")

        que = ""
        char_len = 0
        pos = []
        neg = []
        neu = []

        for sen in sen_list:
            if len(sen_list)-1 == 1: # article is only 1 sentence
                que = sen
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) < 514)): # end of article and can combine
                que = que + sen
                char_len = char_len + len(sen)
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) >= 514)): # end of article and cannot combine
                output = roberta(que) # run que
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = sen # assign last sentence
                char_len = len(sen)
                output = roberta(que) # run last sentence
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif char_len + len(sen) < 514: # combine sentences
                que = que + sen
                char_len = char_len + len(sen)
            else:
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = ""
                que = sen
                char_len = len(sen)

        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        
    df = df.dropna()
        
    return(df)

In [21]:
# marketwatch web scrape NLP
def mwatch_nlp(urls, df):
    
    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    nf = "Story not found"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
                continue
        try:
            master = soup.find(id="maincontent")
            final = master.find("div", class_="article__body article-wrap at16-col16 barrons-article-wrap")
            text = final.text
        except AttributeError:
            pass
        sen_list = text.split(".")

        que = ""
        char_len = 0
        pos = []
        neg = []
        neu = []

        for sen in sen_list:
            if len(sen_list)-1 == 1: # article is only 1 sentence
                que = sen
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) < 514)): # end of article and can combine
                que = que + sen
                char_len = char_len + len(sen)
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) >= 514)): # end of article and cannot combine
                output = roberta(que) # run que
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = sen # assign last sentence
                char_len = len(sen)
                output = roberta(que) # run last sentence
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif char_len + len(sen) < 514: # combine sentences
                que = que + sen
                char_len = char_len + len(sen)
            else:
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = ""
                que = sen
                char_len = len(sen)

        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        
    df = df.dropna()
        
    return(df)

In [22]:
# prnewswire web scrape NLP
def prnw_nlp(urls, df):
    
    df["pos_mean"] = np.nan
    df["pos_median"] = np.nan
    df["pos_min"] = np.nan
    df["pos_max"] = np.nan

    df["neg_mean"] = np.nan
    df["neg_median"] = np.nan
    df["neg_min"] = np.nan
    df["neg_max"] = np.nan

    df["neu_mean"] = np.nan
    df["neu_median"] = np.nan
    df["neu_min"] = np.nan
    df["neu_max"] = np.nan
    
    nf = "Content is currently unavailable"
    
    for i in range(len(urls)): #filter_urls
        URL = urls.iloc[i] # filter_urls
        page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
        soup = BeautifulSoup(page.content, "html.parser")
        if nf in soup.prettify():
                continue
        try:
            master = soup.find(id="main")
            final = master.find("div", class_="col-sm-10 col-sm-offset-1")
        except AttributeError:
            pass
        try:
            text = final.text
        except:
            pass
        text = re.sub('<[^>]+>', '', str(final))
        sen_list = text.split(".")

        que = ""
        char_len = 0
        pos = []
        neg = []
        neu = []

        for sen in sen_list:
            if len(sen_list)-1 == 1: # article is only 1 sentence
                que = sen
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) < 514)): # end of article and can combine
                que = que + sen
                char_len = char_len + len(sen)
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif ((sen_list.index(sen) == len(sen_list)-1) & (char_len + len(sen) >= 514)): # end of article and cannot combine
                output = roberta(que) # run que
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = sen # assign last sentence
                char_len = len(sen)
                output = roberta(que) # run last sentence
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
            elif char_len + len(sen) < 514: # combine sentences
                que = que + sen
                char_len = char_len + len(sen)
            else:
                output = roberta(que)
                neg.append(output[0][0]['score'])
                neu.append(output[0][1]['score'])
                pos.append(output[0][2]['score'])
                que = ""
                que = sen
                char_len = len(sen)

        df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
        df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
        df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
        
    df = df.dropna()
        
    return(df)

In [23]:
# append yahoo, marketwatch, and prnewswire together for entire month dataset
def append_df(df1,df2,df3):
    df = df1.append([df2,df3])
    return(df)

In [25]:
tic = time.perf_counter()
file_paths = load_data()
print("file_paths loaded")
for path in file_paths:
    print(path)
    m_df, p_df, y_df = filter_websites(path)
    print("filter websites")
    m_df = m_df[0:10]
    p_df = p_df[0:10]
    y_df = y_df[0:10]
    
    m_url = m_df.url
    p_url = p_df.url
    y_url = y_df.url
    print("URL")
    
    mwatch_df = mwatch_nlp(m_url, m_df)
    print("mwatch NLP")
    prnw_df = prnw_nlp(p_url, p_df)
    print("prnw NLP")
    yahoo_df = yahoo_nlp(y_url, y_df)
    print("yahoo NLP")
    
    month_df = append_df(yahoo_df,mwatch_df, prnw_df)
    print("Append")
    
    year_number = str(month_df.year.unique().item())
    month_number = str(month_df.month.unique().item())
    datetime_object = datetime.datetime.strptime(month_number, "%m")
    month_name = datetime_object.strftime("%b").lower()
    file_name = 'nlp_'+month_name+str(year_number)+'.csv'
    month_df.to_csv(file_name,
                   index=False)
    
toc = time.perf_counter()
print(f"Completed in {(toc - tic)/60:0.2f} minutes")

file_paths loaded
/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/2021/gdelt_mar2021.csv
filter websites
URL
mwatch NLP
prnw NLP
yahoo NLP
Append
/Users/TyPainter1/Desktop/Masters/spring-2022/capstone/00-data/gdelt_data/2021/gdelt_jan2021.csv
filter websites
URL
mwatch NLP
prnw NLP
yahoo NLP
Append
Completed in 1.78 minutes
